In [2]:
pip install langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [4]:
pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.0 MB/s eta 0:00:00


In [6]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.3 MB/s eta 0:00:00


In [7]:
pip install guidance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.4/255.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00


In [8]:
import torch
import os
import numpy as np

from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from langchain_groq import ChatGroq

In [9]:
# Initialize the language model
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)

In [12]:
def create_prompt_template(template_str, input_variables):
    """
    Create a PromptTemplate object for use with LangChain.

    Args:
        template_str (str): The string template for the prompt.
        input_variables (list): List of input variable names used in the template.

    Returns:
        PromptTemplate: The LangChain PromptTemplate object.
    """
    return PromptTemplate(
        input_variables=input_variables,
        template=template_str
    )


In [13]:
def create_llm_chain(llm, prompt_template):
    """
    Create an LLMChain object for use with LangChain.

    Args:
        llm (ChatGroq): The language model to use.
        prompt_template (PromptTemplate): The prompt template to use.

    Returns:
        LLMChain: The LangChain LLMChain object.
    """
    return LLMChain(
        llm=llm,
        prompt=prompt_template,
        output_key="response",
        verbose=False
    )

In [14]:
# Define prompt templates for different email scenarios
complaint_email_template = """
You are a professional email writer. Respond to the following customer complaint email in a polite and professional manner:
Email: "{customer_email}"
Response:
"""

request_email_template = """
You are a professional email writer. Respond to the following customer request email in a helpful and professional manner:
Email: "{customer_email}"
Response:
"""

feedback_email_template = """
You are a professional email writer. Respond to the following customer feedback email in a courteous and appreciative manner:
Email: "{customer_email}"
Response:
"""

In [15]:
# Create prompt templates for email responses
complaint_prompt_template = create_prompt_template(
    template_str=complaint_email_template,
    input_variables=["customer_email"]
)

request_prompt_template = create_prompt_template(
    template_str=request_email_template,
    input_variables=["customer_email"]
)

feedback_prompt_template = create_prompt_template(
    template_str=feedback_email_template,
    input_variables=["customer_email"]
)

In [16]:
# Create LLMChains for email responses
complaint_chain = create_llm_chain(
    llm=llm,
    prompt_template=complaint_prompt_template
)

request_chain = create_llm_chain(
    llm=llm,
    prompt_template=request_prompt_template
)

feedback_chain = create_llm_chain(
    llm=llm,
    prompt_template=feedback_prompt_template
)

<ipython-input-13-38fdf2b866c8>:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  return LLMChain(


In [17]:
def create_email_response(customer_email, email_type):
    """
    Generate a professional email response based on the type of email.

    Args:
        customer_email (str): The customer's email that needs a response.
        email_type (str): The type of email ("complaint", "request", or "feedback").

    Returns:
        str: The generated email response.
    """
    if email_type == "complaint":
        response = complaint_chain.run(
            {
                "customer_email": customer_email
            }
        )
    elif email_type == "request":
        response = request_chain.run(
            {
                "customer_email": customer_email
            }
        )
    elif email_type == "feedback":
        response = feedback_chain.run(
            {
                "customer_email": customer_email
            }
        )
    else:
        raise ValueError("Invalid email type. Choose from 'complaint', 'request', or 'feedback'.")

    return response

In [18]:
# Example customer emails
complaint_email = "I am very unhappy with the service I received. The product did not meet my expectations and I would like a refund."
request_email = "I would like to inquire about the status of my recent order. Could you provide an update?"
feedback_email = "I just wanted to let you know that I am very satisfied with the product and your service."

# Generate and print email responses for different scenarios
complaint_response = create_email_response(complaint_email, "complaint")
print(f"Complaint email response: {complaint_response}")

request_response = create_email_response(request_email, "request")
print(f"Request email response: {request_response}")

feedback_response = create_email_response(feedback_email, "feedback")
print(f"Feedback email response: {feedback_response}")

<ipython-input-17-a6f039e76ad3>:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = complaint_chain.run(


Complaint email response: Subject: Re: Concern with Recent Purchase and Request for Refund

Dear [Customer's Name],

Thank you for taking the time to share your concerns about the product you received from us. We apologize for not meeting your expectations and appreciate your feedback. We take all complaints seriously and are committed to making things right.

We would be happy to assist you with a refund. To initiate the process, could you please provide us with your order number and a brief explanation of the issue you experienced with the product? This information will help us to expedite your refund and also allow us to review our product and service to prevent similar issues in the future.

If you have already returned the product, please let us know and we will process your refund as soon as possible. If not, we can provide you with a return shipping label and instructions on how to proceed.

Your satisfaction is our top priority, and we value your business. We would like to make